In [8]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import re
from datetime import datetime
from konlpy.tag import Okt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [10]:
# CSV 파일 읽기
df = pd.read_csv("./레이블부여된공지사항(수정본).csv") 
df.head()

,Cid,title,hits,time,cluster
0,1,(예비군연대) 2023년 6월 각 군 현역병 모집계획 안내,116,2023-05-26,43
1,1,2023 세계스카우트잼버리 영외과정활동 「사찰문화체험」 통역 자원봉사자 모집,99,2023-05-26,25
2,1,「대학생 테마별 톡톡 멘토링」 참여 동아리 추가모집,180,2023-05-25,285
3,1,「2023년 국가간 청소년교류」 및 「글로벌 청소년 서밋」청소년 참가자 모집,174,2023-05-25,354
4,1,2023-1 실험실습 교과목 만족도 조사 안내 (드림패스 마일리지 지급),125,2023-05-25,140


In [11]:
# 제목 추출 및 저장
notice_names = df['title'].tolist()
for notice in notice_names:
  print(notice)

(예비군연대) 2023년 6월 각 군 현역병  모집계획 안내
2023 세계스카우트잼버리 영외과정활동 「사찰문화체험」 통역 자원봉사자 모집
「대학생 테마별 톡톡 멘토링」 참여 동아리 추가모집
「2023년 국가간 청소년교류」 및 「글로벌 청소년 서밋」청소년 참가자 모집
2023-1 실험실습 교과목 만족도 조사 안내 (드림패스 마일리지 지급)
[강남열린대학] 인문학특강 개최 홍보
더 안전한 경기도 영상콘텐츠 공모
[참사람사회공헌센터] 한국장학재단 제27기 하계 대학생 재능봉사캠프 모집 안내
[장애학생지원센터]장애인식개선교육 수강 안내
[모두의 연구소] 2023년 AI학교 아이펠 5기 입학생 모집(~6/9)
2023년 여름방학 대학생 교육기부 프로그램 쏙쏙캠프 참여 동아리 모집
[교수학습혁신센터] 2023-2 혁신교수법 적용 교과목 모집 공고
[참사람사회공헌센터] 2023 여름학기 사회봉사교과목 수강신청 안내
동국대학교 장학 홍보대사 단체명 공모전
[외교부] 2024-25년 대한민국 유엔 안보리 진출 제2대 청년사절 접수
2023 어디서든 예술치유 힐링 씨어터
장수군 대표상징물(CI) 디자인 공모전
올담 데이터 활용 시각화 경진대회
그린캠퍼스(게시물 관리) 시행 안내
[5대 핵심역량 함양 프로그램] 이제호 인권 변호사와 함께하는 문화의 다양성과 존중
2023 날씨 빅데이터 콘테스트
기록을 바라보는 여섯 개의 시선
제3회 해양경찰 데이터 활용 아이디어 공모전
2023년 청년인생설계학교 2기 참여자 모집
2023 사이버공격방어대회 참가 안내
웹툰 창작자 발굴 및 육성을 위한 제2회 모든 웹툰 공모전
2023년 산림공공 빅데이터 창업경진대회
제12회 중남미 논문공모전 홍보
2023년 「경기도 청년 노동자 통장」 참여자 모집 안내
[5대 핵심역량 함양 프로그램] 오세훈 서울시장의 동국대 특강
금융감독원 제 11기 대학생 금융교육 봉사단 모집
마약류 이용 성범죄 근절 홍보 콘텐츠 공모전
2023년 사회정책 아이디어 공모전 개최
써라운드 성동-누구나 누리는 기술+문화 홍보
20

In [12]:
# 전처리 함수 정의
def preprocess_text(text):
    # 특수문자 제거
    text = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", text)

    # 숫자 정보 제거
    text = re.sub(r'\d+', '', text)

    # 형태소 분석
    tagger = Okt()
    words = tagger.morphs(text)

    # 불용어 제거
    stop_words = [ '필독', '학기', '학년', '도', '년', '제', '회', '월', '학부', '일', '차', '년도', '안내']  # 불용어 리스트
    words = [word for word in words if word not in stop_words]

    # 분석된 형태소들을 공백으로 결합하여 문장으로 반환
    text = ' '.join(words)

    return text

In [13]:
# 전처리된 공지이름 추출
preprocessed_notice_names = [preprocess_text(notice_name) for notice_name in notice_names]

print("전처리 공지 제목")

for notice in preprocessed_notice_names:
  print(notice)

전처리 공지 제목
예비군 연대 각 군 현 역병 모집 계획
세계 스카우트 잼버리 영 외 과정 활동 사찰 문화 체험 통역 자원봉사자 모집
대학생 테마 별 톡톡 멘토링 참여 동아리 추가 모집
국가 간 청소년 교류 및 글로벌 청소년 서밋 청소년 참가자 모집
실험 실습 교과목 만족도 조사 드림 패스 마일리지 지급
강남 열린 대학 인문학 특강 개최 홍보
더 안전한 경기도 영상 콘텐츠 공모
참사 람 사회 공헌 센터 한국 장학재단 제기 하 계 대학생 재능 봉사 캠프 모집
장애 학생 지 원 센터 장애인 식 개선 교육 수강
모두 의 연구소 AI 학교 아이 펠 기 입학생 모집
여름방학 대학생 교육 기부 프로그램 쏙쏙 캠프 참여 동아리 모집
교수 학습 혁신 센터 혁신 교수 법 적용 교과목 모집 공고
참사 람 사회 공헌 센터 여름 사회 봉사 교과목 수강신청
동국대학교 장학 홍보 대사 단체 명 공모전
외교부 대한민국 유엔 안보리 진출 제대 청년 사절 접수
어디 서든 예술 치유 힐링 씨 어 터
장수군 대표 상징물 CI 디자인 공모전
올담 데이터 활용 시각 화 경진 대회
그린 캠퍼스 게시 물 관리 시행
대 핵심 역량 함양 프로그램 이 제호 인권 변호사 와 함께 하는 문화 의 다양성 과 존중
날씨 빅데이터 콘테스트
기록 을 바라보는 여섯 개 의 시선
해양경찰 데이터 활용 아이디어 공모전
청년 인생 설계 학교 기 참여자 모집
사이버공격 방어 대회 참가
웹툰 창작 자 발굴 및 육성 을 위 한 모든 웹툰 공모전
산림 공공 빅데이터 창업 경진 대회
중남미 논문 공모전 홍보
경기도 청년 노동자 통장 참여자 모집
대 핵심 역량 함양 프로그램 오세훈 서울시장 의 동국대 특강
금융감독원 기 대학생 금융 교육 봉사 단 모집
마약 류 이용 성범죄 근절 홍보 콘텐츠 공모전
사회 정책 아이디어 공모전 개최
써 라운드 성 동 누구 나 누리는 기술 문화 홍보
미지 맵 콘텐츠 크리에이터 모집
통계 데이터 활용 대회
전국 대학생 남북 교류 협력 창업 아이디어 경진 대회 참가
TTable COSMAX 현직 자 강연 에 

In [14]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_notice_names)

In [15]:
lf = df.reindex(columns=["title","cluster"]) 
lf

,title,cluster
0,(예비군연대) 2023년 6월 각 군 현역병 모집계획 안내,43
1,2023 세계스카우트잼버리 영외과정활동 「사찰문화체험」 통역 자원봉사자 모집,25
2,「대학생 테마별 톡톡 멘토링」 참여 동아리 추가모집,285
3,「2023년 국가간 청소년교류」 및 「글로벌 청소년 서밋」청소년 참가자 모집,354
4,2023-1 실험실습 교과목 만족도 조사 안내 (드림패스 마일리지 지급),140
...,...,...
9902,2019년 1학기 학석 공통 과목 개설목록 안내,25
9903,대학원 강좌 수강신청 안내,307
9904,2019학년도 봄 학위수여식 일정 공지,214
9905,2019학년도 1학기 희망강의 신청 안내,169


In [16]:
lf.to_csv(r'.\title cluster dataset.csv', index = False)

# Naive Bayes모델

In [32]:
# CSV 파일 읽기
data = pd.read_csv('title cluster dataset.csv')

In [33]:
#데이터 전처리 테스트가 벡터로 변환
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['title'])
y = data['cluster']

In [34]:
#훈련 데이터 분리 train 데이터 80%, test 데이터 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
#print(X_train)

In [36]:
#모델 훈련
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [37]:
#정확도 구하기
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

정확도: 0.1367305751765893


# SVM모델

In [38]:
from sklearn.svm import SVC

In [39]:
data = pd.read_csv('title cluster dataset.csv')

In [40]:
#데이터 전처리 테스트가 벡터로 변환
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['title'])
y = data['cluster']

In [41]:
#훈련 데이터 분리 train 데이터 80%, test 데이터 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
model = SVC()
model.fit(X_train, y_train)

SVC()

In [43]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

정확도: 0.6316851664984864


# CNN모델

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping


In [18]:
import pandas as pd
# CSV 파일 읽기
data = pd.read_csv('title cluster dataset.csv',dtype = {'cluster' : str})

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9907 entries, 0 to 9906
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    9907 non-null   object
 1   cluster  9907 non-null   object
dtypes: object(2)
memory usage: 154.9+ KB


In [20]:
#전처리
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_notice_names)


tfidf_model = TfidfVectorizer().fit(preprocessed_notice_names)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['cluster'])

In [21]:
print(tfidf_model.vocabulary_) 

{'예비군': 3548, '연대': 3469, '역병': 3446, '모집': 2142, '계획': 1035, '세계': 2902, '스카우트': 3058, '잼버리': 4227, '과정': 1147, '활동': 5583, '사찰': 2685, '문화': 2197, '체험': 4747, '통역': 5040, '자원봉사자': 4135, '대학생': 1718, '테마': 5001, '톡톡': 5031, '멘토링': 2093, '참여': 4646, '동아리': 1818, '추가': 4785, '국가': 1262, '청소년': 4717, '교류': 1192, '글로벌': 1356, '서밋': 2802, '참가자': 4638, '실험': 3207, '실습': 3193, '교과목': 1190, '만족도': 2040, '조사': 4407, '드림': 1856, '패스': 5136, '마일리지': 2015, '지급': 4540, '강남': 873, '열린': 3505, '대학': 1712, '인문학': 3968, '특강': 5080, '개최': 928, '홍보': 5538, '안전한': 3304, '경기도': 988, '영상': 3520, '콘텐츠': 4925, '공모': 1101, '참사': 4642, '사회': 2691, '공헌': 1142, '센터': 2918, '한국': 5361, '장학재단': 4186, '제기': 4373, '재능': 4191, '봉사': 2492, '캠프': 4866, '장애': 4173, '학생': 5335, '장애인': 4174, '개선': 916, '교육': 1207, '수강': 2981, '모두': 2122, '연구소': 3458, 'ai': 15, '학교': 5324, '아이': 3265, '입학생': 4080, '여름방학': 3432, '기부': 1399, '프로그램': 5245, '쏙쏙': 3233, '교수': 1199, '학습': 5343, '혁신': 5496, '적용': 4238, '공고': 1081, '여름': 3431, '수강

In [22]:
print(X)

  (0, 1035)	0.4276328820496505
  (0, 2142)	0.17028977087211436
  (0, 3446)	0.6026999269917589
  (0, 3469)	0.46684895078214067
  (0, 3548)	0.4549025909840208
  (1, 4135)	0.29718286625391055
  (1, 5040)	0.3450465866176246
  (1, 4747)	0.2777760311594997
  (1, 2197)	0.20288488201288005
  (1, 2685)	0.3974090046375888
  (1, 5583)	0.27116265599579575
  (1, 1147)	0.19908248491285327
  (1, 4227)	0.3974090046375888
  (1, 3058)	0.3974090046375888
  (1, 2902)	0.2789876390704115
  (1, 2142)	0.0974911420158906
  (2, 4785)	0.2830645058048866
  (2, 1818)	0.32523917219287873
  (2, 4646)	0.30698405299600645
  (2, 2093)	0.32434134621193256
  (2, 5031)	0.49948175724897215
  (2, 5001)	0.5283869745413193
  (2, 1718)	0.255745542593963
  (2, 2142)	0.14344631581944686
  (3, 4638)	0.1925345767652574
  :	:
  (9900, 1207)	0.20710839521591873
  (9900, 1147)	0.2478229432454879
  (9901, 4734)	1.0
  (9902, 2145)	0.5187177643654213
  (9902, 5338)	0.5187177643654213
  (9902, 1138)	0.4503717629031423
  (9902, 1144)	0.37

In [23]:
print(X.todense()) 

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [24]:
#훈련 데이터 분리 train 데이터 80%, test 데이터 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
print(X_train.shape)

(7925, 5653)


In [26]:
print(y_train.shape)

(7925,)


In [102]:
#CNN모델 생성
model = Sequential()
model.add(Embedding(5000, 128, input_length=100))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [103]:
#모델 훈현
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[EarlyStopping(patience=3)])

InvalidArgumentError: {{function_node __wrapped__SerializeManySparse_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[2] = [0,146] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse]

In [52]:
y_prob = model.predict(X_test, verbose=0) 
y_pred = y_prob.argmax(axis=-1)
accuracy = np.sum(y_pred.flatten() == y_test) / len(y_test)
print("Accuracy:", accuracy)

InvalidArgumentError: {{function_node __wrapped__SerializeManySparse_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[2] = [0,3520] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse]

# LSTM모델

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping


In [54]:
import pandas as pd
# CSV 파일 읽기
data = pd.read_csv('title cluster dataset.csv',dtype = {'cluster' : str})

In [55]:
#전처리
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_notice_names)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['cluster'])

In [56]:
print(X)
print(y)

  (0, 1035)	0.4276328820496505
  (0, 2142)	0.17028977087211436
  (0, 3446)	0.6026999269917589
  (0, 3469)	0.46684895078214067
  (0, 3548)	0.4549025909840208
  (1, 4135)	0.29718286625391055
  (1, 5040)	0.3450465866176246
  (1, 4747)	0.2777760311594997
  (1, 2197)	0.20288488201288005
  (1, 2685)	0.3974090046375888
  (1, 5583)	0.27116265599579575
  (1, 1147)	0.19908248491285327
  (1, 4227)	0.3974090046375888
  (1, 3058)	0.3974090046375888
  (1, 2902)	0.2789876390704115
  (1, 2142)	0.0974911420158906
  (2, 4785)	0.2830645058048866
  (2, 1818)	0.32523917219287873
  (2, 4646)	0.30698405299600645
  (2, 2093)	0.32434134621193256
  (2, 5031)	0.49948175724897215
  (2, 5001)	0.5283869745413193
  (2, 1718)	0.255745542593963
  (2, 2142)	0.14344631581944686
  (3, 4638)	0.1925345767652574
  :	:
  (9900, 1207)	0.20710839521591873
  (9900, 1147)	0.2478229432454879
  (9901, 4734)	1.0
  (9902, 2145)	0.5187177643654213
  (9902, 5338)	0.5187177643654213
  (9902, 1138)	0.4503717629031423
  (9902, 1144)	0.37

In [22]:
#훈련 데이터 분리 train 데이터 80%, test 데이터 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
print(X_train)

  (0, 2058)	0.5295121956531599
  (0, 2850)	0.5295121956531599
  (0, 146)	0.3985605398979046
  (0, 5407)	0.5295121956531599
  (1, 1369)	0.5150583896037998
  (1, 1370)	0.6502426421479397
  (1, 1207)	0.2936305741760656
  (1, 2492)	0.31933637729524744
  (1, 1718)	0.30675797395840376
  (1, 2142)	0.1720589174937572
  (3, 3761)	0.40205031398630325
  (3, 2996)	0.386304955080177
  (3, 2849)	0.326735994377433
  (3, 3016)	0.3327180104118472
  (3, 5349)	0.31374410125612273
  (3, 1893)	0.28499457341071305
  (3, 820)	0.28060428555855077
  (3, 2334)	0.24851870272927307
  (3, 2594)	0.3249006447172236
  (3, 3138)	0.214355626624285
  (4, 2750)	0.5720976589762045
  (4, 3383)	0.46234038028710595
  (4, 5503)	0.3493416714241943
  (4, 2834)	0.22171392483112984
  (4, 5245)	0.3025959731599606
  :	:
  (7921, 244)	0.8240482525800269
  (7921, 2983)	0.5665196178596149
  (7922, 5503)	0.43593056830431165
  (7922, 1719)	0.39561080460896714
  (7922, 4023)	0.43139859904363104
  (7922, 3174)	0.3128045143655545
  (7922, 

In [28]:
#LSTM모델 생성
model = Sequential()
model.add(Embedding(5000, 128, input_length=X.shape[1]))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
#모델훈련
model.fit(scipy.sort_indices X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[EarlyStopping(patience=3)])

SyntaxError: invalid syntax (Temp/ipykernel_10444/993749993.py, line 2)

In [103]:
y_prob = model.predict(X_test, verbose=0) 
y_pred = y_prob.argmax(axis=-1)
accuracy = np.sum(y_pred.flatten() == y_test) / len(y_test)
print("Accuracy:", accuracy)

Accuracy: 0.0005045408678102926
